## Coletor da página de vídeo
Vídeos retornados pelas queries que nos interessam

In [ ]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4
import tqdm
import glob

### Coleta

In [ ]:
df = pd.read_json('parsed_videos.json', lines=True)
df.head()

In [ ]:
links =  df['link'].unique()
len(links)

In [ ]:
url = 'https://www.youtube.com{link}'

for link in links:
    formatted_url = url.format(link=link)
    print(formatted_url)
    response = rq.get(formatted_url)
    
    # pega uma expressão regular (nesse caso tudo que vier depois do sinal de igual na string do link) e pegamos o primeiro
    # grupo de seleção (basicamente o identificador do vídeo)
    link_name = re.search('v=(.*)', link).group(1)
    
    with open('./dados_brutos/video_{}.html'.format(link_name), 'w+') as output:
        output.write(response.text)
    time.sleep(2)

### Processamento dos dados

In [ ]:
with open('parsed_video_info.json', 'w+') as output:
    for video in tqdm.tqdm_notebook(sorted(glob.glob('./dados_brutos/video*'))):
        with open(video, 'r+') as inp:
            page = inp.read()
            parsed = bs4.BeautifulSoup(page, 'html.parser') #html.parser pois estamos processando um arquivo html
            
            #buscando todas as tags que tenham uma determinada class
            class_watch = parsed.find_all(attrs={'class':re.compile(r'watch')})
            id_watch = parsed.find_all(attrs={'id':re.compile(r'watch')}) 
            channel = parsed.find_all('a', attrs={'href':re.compile(r'channel')}) #link do canal
            meta = parsed.find_all('meta') #tags meta que tem algumas infos sobre o vídeo
            
            data = {}
            for element in class_watch:
                column_name = "_".join(element['class']) #valor da classe = nome do atributo nos dados 
                if 'clearfix' in column_name: #clearfix pra não pegar duplicados (característica específica da página)
                    continue
                data[column_name] = element.text.strip() 
                
            for element in id_watch:
                column_name = "_".join(element['id'])
                #if 'clearfix' in column_name: 
                    #continue
                data[column_name] = element.text.strip()
                
            for element in meta:
                column_name = element.get('property')
                if column_name is not None:
                    data[column_name] = element['content']
                    
            for link_number,element in enumerate(channel):
                data['channel_link_{}'.format(link_number)] = element['href']
                
            output.write("{}\n".format(json.dumps(data)))

In [ ]:
element = class_watch[1]
print(element['class'])
print('_'.join(element['class']))
element.text.strip()

In [ ]:
element = meta[4]
print(element)
element['content']

In [ ]:
element = channel[0]
print(element)

### Verificação

In [ ]:
df = pd.read_json('parsed_video_info.json', lines=True)
df.shape

In [ ]:
pd.set_option('display.max_columns', 166)
df.head()

In [ ]:
selected_columns = ['watch-title', 'watch-view-count', 'watch-time-text', 'content-watch-info-tag-list', 'watch7-headline',
                   'watch7-user-header', 'watch8-sentiment-actions', 'og:image', 'og:image:width', 'og:image:height',
                   'og:description', 'og:video:width', 'og:video:height', 'og:video:tag', 'channel_link_0']

In [ ]:
df[selected_columns].head()

In [ ]:
#.feather: compatível com outras linguagens e por ser binário tem um carregamento mais rápido (também é possível
#fazer o carregamento em paralelo)
df[selected_columns].to_feather('raw_data.feather')

In [ ]:
#para posteriormente adicionarmos labels desejados
df[selected_columns].to_csv('raw_data_sem_labels')